In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import chi2_contingency


In [ ]:
cd = pd.read_csv('/kaggle/input/used-cars-dataset-from-cardekhocom/Cardekho_Extract.csv')

In [ ]:
cd

# Data Cleaning

### The data is in the row format and not usable now. We must remove unnessecary data and clean it so we can do EDA

In [ ]:
cd.drop(['Source.Name', 'web-scraper-order', 'web-scraper-start-url','new-price' , 'owner_type'] , axis=1 , inplace=True)

In [ ]:
cd.isnull().sum()

In [ ]:
cd = cd.dropna(subset=['full_name'])

In [ ]:
 cd.full_name = cd.full_name.apply(lambda x : x.split()[0])

In [ ]:
sp = []
for i in cd.selling_price:
    if 'Lakh*' in i:
        sp.append(eval(i.split()[0]))
    elif 'Cr*' in i:
        i=eval(i.split()[0])
        i=i*10000000/100000
        sp.append(i)
    else:
        i = i.replace(",",'')
        i=eval(i.split('*')[0])
        i=i/100000
        sp.append(i)
        
cd.selling_price = sp

In [ ]:
cd.km_driven = cd.km_driven.apply(lambda x : eval(x.replace(",","").split()[0]))

In [ ]:
cd.mileage = cd.mileage.apply(lambda x: x.replace('Mileage' , "").split()[0])
cd.mileage = cd.mileage.apply(lambda x: eval(x))

In [ ]:
engine=[]
for i in cd.engine:
    try:
        result = eval(re.findall(r"[-+]?\d*\.\d+|\d+", i)[0])
        engine.append(result)
    except:
        engine.append(None)
        
cd.engine = engine



In [ ]:
power=[]
for i in cd.max_power:
    try:
        result = eval(re.findall(r"[-+]?\d*\.\d+|\d+", i)[0])
        power.append(result)
    except:
        power.append(None)
        
cd.max_power = power

In [ ]:
seats =[]
for i in cd.seats:
    try:
        result = eval(re.findall(r"[-+]?\d*\.\d+|\d+", i)[0])
        seats.append(result)
    except:
        seats.append(None)
        
cd.seats = seats


In [ ]:
cd.engine.fillna(np.mean(cd.engine) , inplace=True)
cd.seats.fillna(5 , inplace=True)
cd.max_power.fillna(cd.max_power.min() , inplace=True)

# EDA

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(cd.corr() ,  cmap="YlGnBu" , annot=True)

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(y='selling_price',x='fuel_type', data=cd)
plt.title('Fuel Type vs Selling Price');

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(y='engine',x='fuel_type', data=cd)
plt.title('Fuel Type vs Engine');

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(y='engine',x='transmission_type', data=cd)
plt.title("Transmission and Engine cc")

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(y='selling_price',x='transmission_type', data=cd)
plt.title("Transmission and Engine cc")

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(y='engine',x='seats', data=cd)
plt.title("Transmission and Engine cc")

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(y='mileage',x='seats', data=cd)
plt.title("Seats vs  Milage")

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(y='selling_price',x='seats', data=cd)
plt.title("Seats vs  Selling Price");


<u> Here we can that the median price range of 2 seat cars are little high This might be becuase only high performance and 2 seater cars come in 2 seats section. Then median price increases from 4 seater to 7 seater cars and then it decreases from 8 to 10 </u>

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='full_name' , data=cd, )
plt.xticks(rotation=90);
plt.title("Brands and no of items sold");

<u>Here we can see that most sold car brand is maruthi, followed by Hyundai, Honda, Mahindra, Tata and Ford</u>

#### Multicolinerity Check using VIF

In [ ]:
numeric = cd[cd.select_dtypes(include=['number']).columns]

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif["vif_values"] = [variance_inflation_factor(numeric.values , i ) for i in range (numeric.shape[1])]
vif['Variables'] = numeric.columns

In [ ]:
vif

### Coverting Labeled Categorical to Nominal Values

In [ ]:
from sklearn.preprocessing import LabelEncoder
le =LabelEncoder()
cd.full_name = le.fit_transform(cd.full_name)
cd.seller_type = le.fit_transform(cd.seller_type)
cd.fuel_type = le.fit_transform(cd.fuel_type)
cd.transmission_type = le.fit_transform(cd.transmission_type)

In [ ]:
cd

In [ ]:
def remove_outliers(df , col , k):
    mean= df[col].mean()
    sd=df[col].std()
    global df1
    final_list = [x for x in df[col] if (x>mean-k*sd)]
    final_list = [x for x in final_list if (x<mean+k*sd)]
    df1 = df.loc[df[col].isin(final_list)];
    return df1

cd = remove_outliers(cd , 'selling_price' ,1)

# ---------------------------------------------------------------------------------------------------------------

# Linear Regression

### Sampling for linear Regression 

In [ ]:
cd_linear_reg = cd.drop([ 'year','mileage', 'seats', 'engine'] , axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train , test = train_test_split(cd_linear_reg , test_size=0.2)
train_x = train.drop(['selling_price'] , axis=1)
test_x = test.drop(['selling_price'] , axis=1)
train_y = train.selling_price
test_y = test.selling_price

### Linear Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(train_x , train_y)

In [ ]:
pred = lin_reg.predict(test_x)
error = pred- test_y
mse = np.mean(np.square(error))
mae = np.mean(np.abs(error*100/test_y))

In [ ]:
print("MSE :" ,mse)
print("MAPE :", mae)
print("Acuracy :", 100 -mae)

In [ ]:
np.mean(error)

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(error)


In [ ]:
print("Scatter Plot")
plt.figure(figsize=(5,5))
plt.plot(error , "*")
plt.axhline(y=np.mean(error) , color="red")


In [ ]:
print("Regression Plot")
plt.figure(figsize=(5,5))
sns.regplot(test_y , pred )

In [ ]:
from sklearn.linear_model import Lasso , Ridge , ElasticNet

# ---------------------------------------------------------------------------------------------------------------

### Sampling for Lasso 

In [ ]:
train , test = train_test_split(cd , test_size=0.2)
train_x = train.drop(['selling_price'] , axis=1)
test_x = test.drop(['selling_price'] , axis=1)
train_y = train.selling_price
test_y = test.selling_price

## --------------------------------------------------------------------------------------------------------------------------------

# Lasso

In [ ]:
lasso = Lasso()
lasso.fit(train_x , train_y)
pred = lasso.predict(test_x)
error = pred- test_y
mse = np.mean(np.square(error))
mae = np.mean(np.abs(error*100/test_y))
print("MSE :" ,mse)
print("MAPE :", mae)
print("Acuracy :", 100 -mae)

In [ ]:
np.mean(error)

In [ ]:
print("Histogram")
plt.figure(figsize=(10,5))
sns.histplot(error);

In [ ]:
print("Scatter Plot")
plt.figure(figsize=(5,5))
plt.plot(error , "*")
plt.axhline(y=np.mean(error) , color="red");

In [ ]:
print("Regression Plot")
plt.figure(figsize=(5,5))
sns.regplot(test_y , pred );

In [ ]:
coeff_df = pd.DataFrame({
    'Coeficient':lasso.coef_,
    'Columns':train_x.columns
})
coeff_df

### Features selected in Lasso are year, km_driven ,max_power, engine